In [1]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from Models.ActorCritic import ActorCritic
from Models.SoftActorCritic import SoftActorCritic
from Models.DDQN import DoubleDQN

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
torch.cuda.is_available()

True

In [3]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [4]:
env = CustomEnvironment(env_config=env_variables, render_mode="human")
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

0.5


In [5]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 128))

In [6]:
ac_config = open('Environment/ac_config.json')
ac_variables = json.load(ac_config)
ac_config.close

sac_config = open('Environment/sac_config.json')
sac_variables = json.load(sac_config)
sac_config.close

ddqn_config = open('Environment/ddqn_config.json')
ddqn_variables = json.load(ddqn_config)
ddqn_config.close

<function TextIOWrapper.close()>

In [7]:
ddqn_1 = DoubleDQN(ddqn_variables)
# ddqn_2 = DoubleDQN(ddqn_variables)
# ddqn_3 = DoubleDQN(ddqn_variables)
ac = ActorCritic(ac_variables)
# sac = SoftActorCritic(sac_variables)

In [8]:
# models = {i:ddqn_1 for i in range(env.n_agents)}
models = {0: ddqn_1, 1: ac}

In [9]:
# ddqn_1.policy_net.device

In [10]:
def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for i, p in enumerate(named_parameters):
        
        if(p.requires_grad):
            layers.append(i)
            ave_grads.append(p.grad.cpu().abs().mean())
            max_grads.append(p.grad.cpu().abs().max())

    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
    plt.show()

In [11]:
episodes = 50

for episode in tqdm(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",
    # print(curr_obs[0][1])

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for a in env.agents:
            # print(curr_obs[a])
            act, log_prob = models[a].select_action(state = curr_obs[a])
            actions[a] = act
            log_probs[a] = log_prob

        # for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
        #     if i in env.agents:
        #         act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
        #         actions[i] = act
        #         log_probs[i] = log_prob
        #     else:
        #         actions[i] = env.num_actions
        #         log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        # if env.timestep % 100 == 0:
        #     # plot_grad_flow(models[0].policy_net.parameters())
        #     print(rewards)
        

        for id, obs in next_obs.items():
            # print("main shape", obs[0].shape)
            if obs is None:
                print(env.timestep)
                print("next state is None")
            models[id].push_to_buffer(curr_obs[id],actions[id],rewards[id],obs,log_probs[id],terminations[id])
            loss = models[id].update_weights()

            if(terminations[id] == True):
                # loss = models[id].update_weights()
                print(f"Agent {id} died at timestep {env.timestep: 3d}. Current reward {rewards[id]: .3f}.")
                losses[id] = loss


        if env.timestep % 100 == 0:
            for id, agent in env.agents_objects.items():
                # plot_grad_flow(models[0].policy_net.parameters())
                print(f"{id}: {agent.reward: .3f}", end=' ')

            print()
        
        # for i in range(env.n_agents):

        #     if i in env.agents:
        #         env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         env.agents_objects[i].brain1.update_weights() 
            
        #     if i in env.justdie:
        #         env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         loss = env.justdie[i].brain1.update_weights() 
        #         del env.justdie[i]
        #         losses[i] = loss
        
        curr_obs = next_obs
    
    print(f"Episode {episode} end")
    print(len(ddqn_1.replay_buffer))
    # print(f"Losses {losses}")
    print()
    env.close()

  0%|          | 0/50 [00:00<?, ?it/s]

0: -94.422 1: -95.301 
0:  67.108 1: -61.855 
0:  102.847 1: -52.998 
0: -79.216 1: -13.580 
Episode 0 end
492

0: -36.167 1: -37.995 
0: -142.100 1:  21.748 
0: -182.928 1: -83.054 
0: -96.911 1: -170.780 
Episode 1 end
984

0:  20.072 1:  14.527 
0: -66.695 1:  82.405 
0:  4.900 1:  155.495 
0: -69.917 1:  70.190 
Episode 2 end
1476

0: -81.229 1:  14.964 
0: -110.946 1: -53.103 
0: -108.871 1: -122.430 
0: -30.452 1: -88.438 
Episode 3 end
1968

0: -69.821 1: -15.252 
0: -101.385 1:  87.813 
0: -184.500 1: -53.340 
0: -185.960 1:  42.389 
Episode 4 end
2460

0: -91.113 1: -100.847 
0: -76.018 1: -65.979 
0: -37.185 1: -11.606 
0: -104.783 1: -31.540 
Episode 5 end
2952

0: -49.021 1: -99.973 
0: -129.388 1: -131.558 
0: -141.373 1: -46.065 
0: -10.880 1:  42.569 
Episode 6 end
3444

0:  39.924 1:  45.186 
0: -81.954 1:  30.021 
0: -131.027 1: -14.756 
0:  10.252 1: -14.191 
Episode 7 end
3936

0: -4.057 1: -91.337 
0: -2.282 1:  76.188 
0: -88.720 1: -88.320 
0: -107.510 1: -145.560

In [ ]:
plt.imshow(curr_obs[0][0][1].astype(np.int), cmap='gray')

In [ ]:
plt.imshow(curr_obs[0][0][0].astype(np.int), cmap='gray')

In [ ]:
models[0].policy_net((torch.tensor(curr_obs[0][0]), torch.tensor(curr_obs[0][1]), torch.tensor(curr_obs[0][2]), torch.tensor(curr_obs[0][3])))

In [ ]:
models[0].select_action(curr_obs[0])

In [ ]:
models[0].steps_done